### 한강대교 유량 결측치 처리
1. 사용할 컬럼: 자기 자신 -> 청담대교 수위 -> 행주대교 유량 -> 잠수교 수위 -> 청담대교 유량 -> 광진교 유량 -> 한강대교 수위 -> 강화대교 조위 -> 방류량 -> 강수량... -> 행주대교 수위
2. 그런데 이 순서는 모델의 행동을 설명하는 것이지 인과관계를 반드시 보장한다고 할 수는 없음...

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings(action='ignore')

import time
from glob import glob
from tqdm import tqdm

import tensorflow as tf
import tensorflow.keras as keras
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras import backend as K
# from keras.utils import plot_model

from keras.layers import Activation, Dense, LSTM, GRU, Bidirectional, AveragePooling1D, GlobalAveragePooling1D, Dropout, Flatten, BatchNormalization, TimeDistributed, Input, Reshape, RepeatVector, Concatenate, GlobalMaxPooling1D 
from keras.layers.convolutional import Conv1D, MaxPooling1D, AveragePooling1D


#GPU 자원이 부족
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
import os
os.chdir('/content/drive/MyDrive/신입기수 프로젝트/Dataset/')

In [4]:
# 결측치 처리한 수위4개+유량1개 데이터셋 가져오기
raw_data = pd.read_csv('Copy of 0827 광진행주유량수위x4.csv', encoding = 'cp949')

# 6912개 수위데이터 + 한강대교_수위 데이터셋 가져오기
newdata5 = pd.read_csv('newdata5.csv', encoding = 'cp949')

# 방류량 컬럼 추가
raw_data = pd.concat([raw_data['광진교_수위'], newdata5['방류량']], axis = 1)

In [5]:
raw_data

,광진교_수위,방류량
0,1.20,575.70
1,1.20,575.30
2,1.20,555.00
3,1.20,575.70
4,1.20,575.40
...,...,...
276331,1.12,258.67
276332,1.12,258.58
276333,1.12,258.56
276334,1.12,258.65


In [7]:
x = raw_data

In [8]:
x

,광진교_수위,방류량
0,1.20,575.70
1,1.20,575.30
2,1.20,555.00
3,1.20,575.70
4,1.20,575.40
...,...,...
276331,1.12,258.67
276332,1.12,258.58
276333,1.12,258.56
276334,1.12,258.65


In [9]:
# 정규화
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

input_x_regular = x.copy()

for i in range(len(x.columns)):
    scaler = MinMaxScaler()                       #StandardScaler() , MinMaxScaler() , RobustScaler()
    a=np.array(input_x_regular.iloc[:,i])
    tmp_x = scaler.fit_transform(a.reshape(-1,1))
    input_x_regular.iloc[:,i] = tmp_x

# 원하는 칼럼명 입력하기

In [10]:
#ydata = raw_data[['현재수위','유입량','저수량','공용량','방류량','강화대교_조위','청담대교_유량','잠수교_유량','한강대교_유량']]
y = raw_data['방류량']

# 원하는 칼럼명 입력하기

In [11]:
input_regular = pd.concat([input_x_regular, y], axis = 1)
input_regular

,광진교_수위,방류량,방류량
0,0.056452,0.058462,575.70
1,0.056452,0.058447,575.30
2,0.056452,0.057689,555.00
3,0.056452,0.058462,575.70
4,0.056452,0.058451,575.40
...,...,...,...
276331,0.045699,0.046624,258.67
276332,0.045699,0.046621,258.58
276333,0.045699,0.046620,258.56
276334,0.045699,0.046624,258.65


# 원하는 칼럼명 입력하기

In [12]:
df_nanindex = pd.read_csv('/content/drive/MyDrive/신입기수 프로젝트/Dataset/rawdata.csv')

nanIndex = df_nanindex[df_nanindex['방류량'].isnull() == True].index

In [13]:
nanIndex

Int64Index([ 13200,  68081,  68083,  68084,  68085,  68086,  68087,  68089,
             68090,  68091,
            ...
            270533, 270534, 271588, 272215, 272216, 272217, 272218, 272219,
            272220, 272221],
           dtype='int64', length=743)

In [14]:
from tqdm import tqdm

xdata = []
ydata = []
xdata_idx = []
ydata_idx = [-1]
for i in range(len(input_regular.columns)-1):
    xdata_idx.append(i)

w =  6 #window size

idx = []
for i in range(12):
  idx.append(i*26496)

for i in tqdm(range(11)):
    if i != 10:
        tmp = input_regular[idx[i]:idx[i+1]]
    else:
        tmp = input_regular[idx[i]:]

    for j in range(len(tmp)-w):
        xdata.append(np.array(tmp.iloc[j:j + w, xdata_idx]).astype(float))

    for j in range(w,len(tmp)):
        ydata.append(np.array(tmp.iloc[j, ydata_idx]).astype(float))

100%|██████████| 11/11 [02:53<00:00, 15.73s/it]


In [15]:
print(np.array(xdata).shape)
print(np.array(ydata).shape)

(276270, 6, 2)
(276270, 1)


In [16]:
# 나중에 학습된 모델에 이거 넣어서 predict해서 결측치 채우기
x_real_predict = []
y_real_predict = []

for i in range(len(nanIndex)):
    k = nanIndex[-(i+1)]
    
    x_real_predict.insert(0,(xdata[k - w * (k // 26496 + 1)]))
    y_real_predict.insert(0,(ydata[k - w * (k // 26496 + 1)]))
    
    xdata.pop(k - w * (k // 26496 + 1))
    ydata.pop(k - w * (k // 26496 + 1))

In [17]:
print(np.array(x_real_predict).shape)
print(np.array(y_real_predict).shape)

(743, 6, 2)
(743, 1)


In [18]:
print(np.array(xdata).shape)
print(np.array(ydata).shape)

(275527, 6, 2)
(275527, 1)


In [19]:
pd.DataFrame(ydata).isnull().sum()

0    0
dtype: int64

In [20]:
xtrain = np.array(xdata[:-50000])
ytrain = np.array(ydata[:-50000])

xtest = np.array(xdata[-50000:])
ytest = np.array(ydata[-50000:])

In [21]:
print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)
print(ytest.shape)

(225527, 6, 2)
(225527, 1)
(50000, 6, 2)
(50000, 1)


## 모델 생성

In [22]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

if 'model_enc_dec_cnn' in locals(): 
    del model_enc_dec_cnn 
if 'history' in locals():
    del history

optimizer = tf.keras.optimizers.Adam(lr=0.01) #기본값 0.001

model_enc_dec_cnn = Sequential()
model_enc_dec_cnn.add(Conv1D(filters=64, kernel_size=2, padding = 'causal', activation='relu', input_shape=(w, xtrain.shape[2])))
#model_enc_dec_cnn.add(Conv1D(filters=64, kernel_size=2, padding = 'causal', activation='relu'))
#model_enc_dec_cnn.add(MaxPooling1D(pool_size=2))
model_enc_dec_cnn.add(Flatten())

model_enc_dec_cnn.add(RepeatVector(1))
model_enc_dec_cnn.add(LSTM(200, activation='relu', return_sequences=True))
model_enc_dec_cnn.add(LSTM(200, activation='relu', return_sequences=True))
# model_enc_dec_cnn.add(LSTM(200, activation='relu', return_sequences=True))
# model_enc_dec_cnn.add(LSTM(200, activation='relu', return_sequences=True))
# model_enc_dec_cnn.add(LSTM(200, activation='relu', return_sequences=True))
model_enc_dec_cnn.add(LSTM(200, activation='relu'))

model_enc_dec_cnn.add(Dense(100, activation='relu'))
model_enc_dec_cnn.add(Dense(1))

model_enc_dec_cnn.compile(loss=root_mean_squared_error, optimizer=optimizer, metrics =["accuracy"])

model_enc_dec_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 6, 64)             320       
                                                                 
 flatten (Flatten)           (None, 384)               0         
                                                                 
 repeat_vector (RepeatVector  (None, 1, 384)           0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 1, 200)            468000    
                                                                 
 lstm_1 (LSTM)               (None, 1, 200)            320800    
                                                                 
 lstm_2 (LSTM)               (None, 200)               320800    
                                                        

In [23]:
# plot_model(model=model_enc_dec_cnn, show_shapes=True)

In [24]:
epochs = 100
batch_size = xtrain.shape[0] // 1000
validation = 0.2

checkpoint_callback = ModelCheckpoint(filepath='best_model.h5', save_weights_only=False, monitor='val_loss',
                                         mode='min', save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', min_delta=0.005, patience=10, mode='min')
rlrop_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.5, mode='min', patience=3, min_lr=0.001)

In [25]:
## 학습

history = model_enc_dec_cnn.fit(xtrain, ytrain, verbose=1, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, checkpoint_callback, rlrop_callback])

Epoch 1/100
802/802 [==============================] - 42s 48ms/step - loss: 126.6211 - accuracy: 0.0000e+00 - val_loss: 43.8954 - val_accuracy: 0.0000e+00 - lr: 0.0100
Epoch 2/100
802/802 [==============================] - 37s 47ms/step - loss: 87.1508 - accuracy: 0.0000e+00 - val_loss: 32.4168 - val_accuracy: 0.0000e+00 - lr: 0.0100
Epoch 3/100
802/802 [==============================] - 38s 47ms/step - loss: 81.7501 - accuracy: 0.0000e+00 - val_loss: 36.9272 - val_accuracy: 0.0000e+00 - lr: 0.0100
Epoch 4/100
802/802 [==============================] - 37s 47ms/step - loss: 80.0633 - accuracy: 0.0000e+00 - val_loss: 38.2468 - val_accuracy: 0.0000e+00 - lr: 0.0100
Epoch 5/100
802/802 [==============================] - 38s 47ms/step - loss: 77.8375 - accuracy: 0.0000e+00 - val_loss: 30.3090 - val_accuracy: 0.0000e+00 - lr: 0.0100
Epoch 6/100
802/802 [==============================] - 38s 47ms/step - loss: 75.3171 - accuracy: 0.0000e+00 - val_loss: 30.2660 - val_accuracy: 0.0000e+00 - lr

In [26]:
## 평가

keras_model_best = keras.models.load_model('best_model.h5', custom_objects={'root_mean_squared_error': root_mean_squared_error})

yhat = keras_model_best.predict(xtest, verbose=1)

yhat

keras_model_best.evaluate(xtest,ytest)

ValueError: ignored

In [ ]:
## 학습 과정 시각화

fig, loss_ax = plt.subplots(1, 2)
fig.set_size_inches((15,5))

loss_ax[0].plot(history.history['loss'], 'b', label='train loss')
loss_ax[0].plot(history.history['val_loss'], 'r', label='val loss')

loss_ax[1].plot(history.history['accuracy'], 'y', label='train acc')
loss_ax[1].plot(history.history['val_accuracy'], 'g', label='val acc')

loss_ax[0].set_xlabel('epoch')
loss_ax[0].set_ylabel('loss')
loss_ax[1].set_xlabel('epoch')
loss_ax[1].set_ylabel('accuray')

loss_ax[0].legend(loc='upper right')
loss_ax[1].legend(loc='lower right')

plt.show()

In [ ]:
## 결과 시각화

tl_predict = np.array(yhat)

#결과 시각화
plt.figure(figsize=(15,10))
plt.plot(ytest, marker = ",", label = "actual")
plt.plot(tl_predict, marker = ",", label="predict", alpha = 0.5)
plt.legend(prop={'size':10})  

plt.show()

In [ ]:
## 점수 확인
print("RMSE : ",float(root_mean_squared_error(ytest, tl_predict)))

## 파일 저장

In [ ]:
x_real_predict = np.array(x_real_predict)
y_real_hat = keras_model_best.predict(x_real_predict, verbose = 1)

In [ ]:
for i in range(len(nanIndex)):
    raw_data.iloc[nanIndex[i]] = np.float64(y_real_hat[i])

In [ ]:
raw_data['방류량'].to_csv('/content/drive/MyDrive/신입기수 프로젝트/Dataset/newdata7_방류량_광진교_포함.csv', index = False, encoding = 'cp949')